### Module import
In this code section the required modules are imported:
* __*date*__ to create a simulated NPI project. `Project`class requires milestone dates in `Date` class (format).
* __*matplotlib*__ to plot the fuzzy functions.
* __*numpy*__ to work with values in fuzzy variables.
* __*simulation*__ is a custom module with the required classes to simulate the NPI Sourcing process. The code can be found in `simulation.py` or `simulation.ipynb`.
* __*skfuzzy*__ is a fuzzy logic framework for Python.

In [ ]:
from datetime import date
from math import ceil, floor
from statistics import mean, stdev
import matplotlib.pyplot as plt
import numpy as np
import simulation
import skfuzzy as fuzzy

## Generate training data frame

### Generate sample suppliers
Based in fuzzy variables (delivery time, quotation time, price, punctuality) and their values (low, regular, high), it will be necessary to generate 81 suppliers with different profiles each one, aiming to obtain a initial data frame with unbiased data.
Supplier/profile quantity is calculated as follows in this formula: $`R_{max} = \prod_{i=1}^{n} m_{i} = \prod_{i=1}^{4} 3 = 3^{4} = 81`$

In [ ]:
env = simulation.Environment()  # Initialize environment for the simulation.

current_supplier_names = ("Tuberías ABC, S.A. de C.V.", "Tuberías DEF, S.A.", "Tuberías GHI, S. de R.L. de C.V.")
current_Supplier_ids = []
current_supplier_profile = (("regular", "low", "high", "high"), ("regular", "high", "low", "regular"), ("high", "high", "low", "low"))

for delivery_time in ["low", "regular", "high"]:
	for quotation_time in ["low", "regular", "high"]:
		for price in ["low", "regular", "high"]:
			for punctuality in ["low", "regular", "high"]:
				supplier_number = len(env.suppliers) + 1
				supplier_id = f"1{str(supplier_number).zfill(7)}"    # Supplier ID must be 8 character length string.

				if (delivery_time, quotation_time, price, punctuality) in current_supplier_profile:
					supplier_index = current_supplier_profile.index((delivery_time, quotation_time, price, punctuality))
					supplier_name = current_supplier_names[supplier_index]
					print(f"{supplier_name} has ID {supplier_id}.")
					current_Supplier_ids.append(supplier_id)
				else:
					supplier_name = f"Supplier {supplier_number}"   # Generate a supplier name concatenating Supplier and supplier number.

				supplier = simulation.Supplier(supplier_id, supplier_name, delivery_time, quotation_time, price, punctuality)
				env.add_supplier(supplier)  # Add supplier to simulation environment.

In [ ]:
estrella = simulation.Supplier("10000082", "Súper Fantástico, S.A.S.", "low", "low", "low", "high")
env.add_supplier(estrella)

## Generate current suppliers

### Create project and ECNs
Create two sample projects named *Alaska* and *Pandora*.
It is important to define the milestone dates for these projectss. Required milestone dates are:
* **Design freeze (DF):** Restrict not important changes to the design of the project. Most of ECNs (Engineering Change Notification) are released near this date.
* **MCS:** I do not remember what does it stands for in this moment. In this stage a prototype containing the exact sourced material for the project to review to review their feasibility and adjust parameters in case it is needed. In this stage, part numbers do not need to be PPAP approved and built units cannot be sold.
* **Pilot:** In this stage a small but significant quantity of units of the project are produced. It aims to test the whole manufacturing process. All the material needs to be PPAP approved in this stage. Pilot units may be sold under Marketing approval.
* **Start of production (SOP):** Project implementation. Suppliers need to be ready at least six weeks before this date, however, *supplier readiness* may be longer depending on material lead time.

A quantity of 300 ECNs is created for this initial data frame. Each ECN contains a variable quantity of part numbers of different kinds. The quantity and kinds are randomly generated taking into account real statistics.

The simulated part numbers are copper tubes. From a real data set it was determined six kinds three complexity degrees (low, medium, regular). Both parameters affect prices and quantities.

In [ ]:
alaska = simulation.Project(
    name="Alaska",
    df_date=date(2024,5,10), 
    mcs_date=date(2024,7,15),
    pilot_date=date(2024,9,1), 
    sop_date=date(2025,1,27)
)   # Create project Alaska.

pandora = simulation.Project(
    name="Pandora",
    df_date=date(2025,8,19),
    mcs_date=date(2026,2,5),
    pilot_date=date(2026,4,23),
    sop_date=date(2026,6,26)
)   # Create project Pandora.

env.gen_ecns(alaska, 100)   # Create 100 ECNs for Alaska project.
env.gen_ecns(pandora, 200)  # Create 200 ECNs for Pandora project.

### Generating data frame
Generate the data frame is divided in two steps:
1. Quoting all the ECNs with all the suppliers. The simulation uses real statistical data to generate random dates and prices.
2. Implementing each ECN with random chosen suppliers. A uniform distribution is used to randomly chose the suppliers.

In [ ]:
env.quote_all_ecns()
env.gen_initial_item_master_df()

env.item_master # Display all the dataframe, with no filters.

## Generate data frame for fuzzy model
In this section a new project and data frame containing 10 copper piping ECNs is generated to apply it with the 

In [ ]:
juneau = simulation.Project(
    name="Juneau",
    df_date=date(2024,4,9),
    mcs_date=date(2024,6,17),
    pilot_date=date(2024,8,1),
    sop_date=date(2024,11,25)
)

env.gen_ecns(juneau, 10)

In [ ]:
for supplier in env.suppliers:
    if supplier.id not in current_Supplier_ids:
        env.deactivate_supplier(supplier)

env.activate_supplier(estrella)

for ecn in juneau.ecns:
    env.quote_ecn_all_suppliers(ecn)

env.item_master[env.item_master["Project"] == "Juneau"]

## Define parameters for fuzzy variables

In [ ]:
def get_µσ_punctuality(profile: str):
    punctuality_list = []

    for supplier_name in env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Punctuality profile"] == profile) & (env.item_master["OTD"] == True)]["Supplier name"].unique():
        punctuality = len(env.item_master[(env.item_master["Supplier name"] == supplier_name) & (env.item_master["Awarded"] == True) & (env.item_master["OTD"] == True)]) / len(env.item_master[(env.item_master["Supplier name"] == supplier_name) & (env.item_master["Awarded"] == True)])

        punctuality_list.append(punctuality)

    return (mean(punctuality_list), stdev(punctuality_list))

In [ ]:
max_delivery_time = round(env.item_master[(env.item_master["Awarded"] == True)]["Delivery time"].max())
max_quotation_time = round(env.item_master[(env.item_master["Awarded"] == True)]["Quotation time"].max())
µ_business_total = float(env.item_master[(env.item_master["Project"] == "Juneau")][["Supplier name", "FY Spend"]].groupby("Supplier name").sum().mean())
σ_business_total = float(env.item_master[(env.item_master["Project"] == "Juneau")][["Supplier name", "FY Spend"]].groupby("Supplier name").sum().std())
min_business_total = µ_business_total - 3*σ_business_total
max_business_total = µ_business_total + 3*σ_business_total

var_delivery_time = np.arange(0, max_delivery_time + 1, 1)
var_quotation_time = np.arange(0, max_quotation_time + 1, 1)
var_price = np.arange(floor(µ_business_total - 3*σ_business_total), ceil(µ_business_total + 3*σ_business_total), 0.01)
var_punctuality = np.arange(0, 2, 0.01)
var_supplier = np.arange(0, 11, 0.01)

In [ ]:
µ_delivery_time_low = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Delivery profile"] == "low")]["Delivery time"].mean())
σ_delivery_time_low = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Delivery profile"] == "low")]["Delivery time"].std())
µ_delivery_time_medium = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Delivery profile"] == "regular")]["Delivery time"].mean())
σ_delivery_time_medium = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Delivery profile"] == "regular")]["Delivery time"].std())
µ_delivery_time_high = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Delivery profile"] == "high")]["Delivery time"].mean())
σ_delivery_time_high = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Delivery profile"] == "high")]["Delivery time"].std())

µ_quotation_time_low = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Quotation profile"] == "low")]["Quotation time"].mean())
σ_quotation_time_low = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Quotation profile"] == "low")]["Quotation time"].std())
µ_quotation_time_medium = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Quotation profile"] == "regular")]["Quotation time"].mean())
σ_quotation_time_medium = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Quotation profile"] == "regular")]["Quotation time"].std())
µ_quotation_time_high = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Quotation profile"] == "high")]["Quotation time"].mean())
σ_quotation_time_high = round(env.item_master[(env.item_master["Awarded"] == True) & (env.item_master["Quotation profile"] == "high")]["Quotation time"].std())

µ_punctuality_low, σ_punctuality_low = get_μσ_punctuality("low")
µ_punctuality_medium, σ_punctuality_medium = get_μσ_punctuality("regular")
µ_punctuality_high, σ_punctuality_high = get_μσ_punctuality("high")

In [ ]:
delivery_time_low = fuzzy.trapmf(var_delivery_time, [0, 0, µ_delivery_time_low, µ_delivery_time_low + σ_delivery_time_low])
delivery_time_medium = fuzzy.trimf(var_delivery_time, [µ_delivery_time_medium - σ_delivery_time_medium, µ_delivery_time_medium, µ_delivery_time_medium + σ_delivery_time_medium])
delivery_time_high = fuzzy.trapmf(var_delivery_time, [µ_delivery_time_high - σ_delivery_time_high, µ_delivery_time_high, max_delivery_time, max_delivery_time])

quotation_time_low  = fuzzy.trapmf(var_quotation_time, [0, 0, µ_quotation_time_low, µ_quotation_time_low + σ_quotation_time_low])
quotation_time_medium  = fuzzy.trimf(var_quotation_time, [µ_quotation_time_medium - σ_quotation_time_medium, µ_quotation_time_medium, µ_quotation_time_medium + σ_quotation_time_medium])
quotation_time_high = fuzzy.trapmf(var_quotation_time, [µ_quotation_time_high - σ_quotation_time_high, µ_quotation_time_high, max_quotation_time, max_quotation_time])

punctuality_low = fuzzy.trapmf(var_punctuality, [0, 0, round(µ_punctuality_low, 2), round(µ_punctuality_low + σ_punctuality_low, 2)])
punctuality_medium = fuzzy.trimf(var_punctuality, [round(µ_punctuality_medium - σ_punctuality_medium, 2), round(µ_punctuality_medium, 2), round(µ_punctuality_medium + σ_punctuality_medium, 2)])
punctuality_high = fuzzy.trapmf(var_punctuality, [round(µ_punctuality_high - σ_punctuality_high, 2), round(µ_punctuality_high, 2), 1, 1])

price_low = fuzzy.trapmf(var_price, [floor(min_business_total), floor(min_business_total), round(µ_business_total - σ_business_total), round(μ_business_total)])
price_medium = fuzzy.trimf(var_price, [round(μ_business_total - σ_business_total), round(μ_business_total), round(μ_business_total + σ_business_total)])
price_high = fuzzy.trapmf(var_price, [round(μ_business_total), round(μ_business_total + σ_business_total), ceil(max_business_total), ceil(max_business_total)])

supplier_low = fuzzy.trimf(var_supplier, [0, 2.5, 5])
supplier_medium = fuzzy.trimf(var_supplier, [2.5, 5, 7.5])
supplier_high = fuzzy.trimf(var_supplier, [5, 7.5, 10])

In [ ]:
fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(nrows=5, figsize=(8, 9))

ax0.plot(var_delivery_time, delivery_time_low, "b", linewidth=1.5, label="Good")
ax0.plot(var_delivery_time, delivery_time_medium, "g", linewidth=1.5, label="Regular")
ax0.plot(var_delivery_time, delivery_time_high, "r", linewidth=1.5, label="Bad")
ax0.set_title("Delivery time")
ax0.legend()

ax1.plot(var_quotation_time, quotation_time_low, "b", linewidth=1.5, label="Good")
ax1.plot(var_quotation_time, quotation_time_medium, "g", linewidth=1.5, label="Regular")
ax1.plot(var_quotation_time, quotation_time_high, "r", linewidth=1.5, label="Bad")
ax1.set_title("Quotation time")
ax1.legend()

ax2.plot(var_punctuality, punctuality_low, "b", linewidth=1.5, label="Bad")
ax2.plot(var_punctuality, punctuality_medium, "g", linewidth=1.5, label="Regular")
ax2.plot(var_punctuality, punctuality_high, "r", linewidth=1.5, label="Good")
ax2.set_title("Punctuality")
ax2.legend()

ax3.plot(var_price, price_low, "b", linewidth=1.5, label="Low")
ax3.plot(var_price, price_medium, "g", linewidth=1.5, label="Regular")
ax3.plot(var_price, price_high, "r", linewidth=1.5, label="High")
ax3.set_title("Price")
ax3.legend()

ax4.plot(var_supplier, supplier_low, "b", linewidth=1.5, label="Bad")
ax4.plot(var_supplier, supplier_medium, "g", linewidth=1.5, label="Regular")
ax4.plot(var_supplier, supplier_high, "r", linewidth=1.5, label="Good")
ax4.set_title("Supplier")
ax4.legend()

for ax in [ax0, ax1, ax2, ax3, ax4]:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

plt.tight_layout()
plt.show()

In [ ]:
evaluation_supplier_id = "10000073"

crisp_delivery_time = env.item_master[(env.item_master["Supplier ID"] == evaluation_supplier_id) & (env.item_master["Awarded"] == True)]["Delivery time"].mean()
crisp_quotation_time = env.item_master[(env.item_master["Supplier ID"] == evaluation_supplier_id) & (env.item_master["Awarded"] == True)]["Quotation time"].mean()
crisp_price = env.item_master[(env.item_master["Project"] == "Juneau") & (env.item_master["Supplier ID"] == evaluation_supplier_id)]["FY Spend"].sum()
crisp_punctuality = len(env.item_master[(env.item_master["Supplier ID"] == evaluation_supplier_id) & (env.item_master["Awarded"] == True) & (env.item_master["OTD"] == True)]) / len(env.item_master[(env.item_master["Supplier ID"] == evaluation_supplier_id) & (env.item_master["Awarded"] == True)])

# Assign membership degree

price_level_low = fuzzy.interp_membership(var_price, price_low, crisp_price)
price_level_medium = fuzzy.interp_membership(var_price, price_medium, crisp_price)
price_level_high = fuzzy.interp_membership(var_price, price_high, crisp_price)

punctuality_level_low = fuzzy.interp_membership(var_punctuality, punctuality_low, crisp_punctuality)
punctuality_level_medium = fuzzy.interp_membership(var_punctuality, punctuality_medium, crisp_punctuality)
punctuality_level_high = fuzzy.interp_membership(var_punctuality, punctuality_high, crisp_punctuality)

delivery_time_level_low = fuzzy.interp_membership(var_delivery_time, delivery_time_low, crisp_delivery_time)
delivery_time_level_medium = fuzzy.interp_membership(var_delivery_time, delivery_time_medium, crisp_delivery_time)
delivery_time_level_high = fuzzy.interp_membership(var_delivery_time, delivery_time_high, crisp_delivery_time)

quotation_time_level_low = fuzzy.interp_membership(var_quotation_time, quotation_time_low, crisp_quotation_time)
quotation_time_level_medium = fuzzy.interp_membership(var_quotation_time, quotation_time_medium, crisp_quotation_time)
quotation_time_level_high = fuzzy.interp_membership(var_quotation_time, quotation_time_high, crisp_quotation_time)

# Rule application
# Example code uses np.fmax for OR operator. I will use np.fmin for AND.
rule_1 = np.fmin.reduce([price_level_low, punctuality_level_high, delivery_time_level_low, quotation_time_level_low])

rule_2 = np.fmin.reduce([price_level_medium, punctuality_level_high, delivery_time_level_low, quotation_time_level_low])

rule_3 = np.fmin.reduce([price_level_low, punctuality_level_medium, delivery_time_level_low, quotation_time_level_low])

rule_4 = np.fmin.reduce([price_level_low, punctuality_level_high, delivery_time_level_medium, quotation_time_level_low])

rule_5 = np.fmin.reduce([price_level_low, punctuality_level_high, delivery_time_level_low, quotation_time_level_medium])

rule_6 = np.fmin.reduce([price_level_high, punctuality_level_low, delivery_time_level_high, quotation_time_level_high])

rule_7 = np.fmin.reduce([price_level_medium, punctuality_level_low, delivery_time_level_high, quotation_time_level_high])

rule_8 = np.fmin.reduce([price_level_high, punctuality_level_medium, delivery_time_level_high, quotation_time_level_high])

rule_9 = np.fmin.reduce([price_level_high, punctuality_level_low, delivery_time_level_medium, quotation_time_level_high])

rule_10 = np.fmin.reduce([price_level_high, punctuality_level_low, delivery_time_level_high, quotation_time_level_medium])

rule_11 = 1 - np.fmax.reduce([rule_1, rule_2, rule_3, rule_4, rule_5, rule_6, rule_7, rule_8, rule_9, rule_10])

supplier_activation_low = np.fmin(np.fmax.reduce([rule_3, rule_4, rule_5, rule_6, rule_10]), supplier_low)
supplier_activation_medium = np.fmin(rule_11, supplier_medium)
supplier_activation_high = np.fmin(np.fmax.reduce([rule_1, rule_2, rule_7, rule_8, rule_9]), supplier_high)

supplier_0 = np.zeros_like(var_supplier)

aggregated = np.fmax.reduce([supplier_activation_low, supplier_activation_medium, supplier_activation_high])


# Defuzzification
supplier_score = fuzzy.defuzz(var_supplier, aggregated, "centroid")
supplier_activation = fuzzy.interp_membership(var_supplier, aggregated, supplier_score)

In [ ]:
fig, (ax5, ax6) = plt.subplots(nrows=2, figsize=(8, 9))

ax5.fill_between(var_supplier, supplier_0, supplier_activation_low, facecolor="b", alpha=0.7)
ax5.plot(var_supplier, supplier_low, "b", linewidth=0.5, linestyle="--")
ax5.fill_between(var_supplier, supplier_0, supplier_activation_medium, facecolor="g", alpha=0.7)
ax5.plot(var_supplier, supplier_medium, "g", linewidth=0.5, linestyle="--")
ax5.fill_between(var_supplier, supplier_0, supplier_activation_high, facecolor="r", alpha=0.7)
ax5.plot(var_supplier, supplier_high, "r", linewidth=0.5, linestyle="--")
ax5.set_title("Output membership")

ax6.plot(var_supplier, supplier_low, "b", linewidth=0.5, linestyle="--")
ax6.plot(var_supplier, supplier_medium, "g", linewidth=0.5, linestyle="--")
ax6.plot(var_supplier, supplier_high, "r", linewidth=0.5, linestyle="--")
ax6.fill_between(var_supplier, supplier_0, aggregated, facecolor="Orange", alpha=0.7)
ax6.plot([supplier_score, supplier_score], [0, supplier_activation], "k", linewidth=1.5, alpha=0.9)
ax6.set_title("Aggregated membership and supplier score (line)")

for ax in [ax5, ax6]:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()

plt.tight_layout()
plt.show()

In [ ]:
supplier_score

In [ ]:
print("Rules:")
for i, rule in enumerate([rule_1, rule_2, rule_3, rule_4, rule_5, rule_6, rule_7, rule_8, rule_9, rule_10, rule_11]):
    print(f"Rule {i+1}: {rule:.3f}")

print("\nSupplier activations:")
print(f"Low: {supplier_activation_low.max():.3f}")
print(f"Medium: {supplier_activation_medium.max():.3f}")
print(f"High: {supplier_activation_high.max():.3f}")

In [ ]:
testing_supplier = env.get_supplier("id", "10000073")
otd_sop = []

for i in range(100):
    for ecn in juneau.ecns:
        env.implement_ecn(ecn, testing_supplier, overwrite=True)
    
    otd = len(env.item_master[(env.item_master["Project"] == "Juneau") & (env.item_master["Supplier ID"] == testing_supplier.id) & (env.item_master["Awarded"] == True) & (env.item_master["SOP ready"] == True)]) / len(env.item_master[(env.item_master["Project"] == "Juneau") & (env.item_master["Supplier ID"] == testing_supplier.id) & (env.item_master["Awarded"] == True)])

    otd_sop.append(otd)

mean(otd_sop)